In [6]:
api_blocks = []

block_begin = False
cache = ''
for line in open('./public.md', 'r', encoding='utf-8'):
    if line.startswith('## '):
        block_begin = True
        
        if cache.__len__() > 0:
            api_blocks.append(cache)
            cache = ''
    
    if block_begin:
        cache += line

if cache.__len__() > 0:
    api_blocks.append(cache)
        

In [32]:
import re

# 读取 Markdown 文件内容
def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 解析 Markdown 内容
def parse_markdown(content):
    lines = content.split('\n')
    api_name = ''
    response_interface = []
    is_response_section = False
    for line in lines:
        line = line.strip()

        # 提取 API 名称
        if line.startswith('## `'):
            api_name = re.search(r'`([^`]+)`', line).group(1)

        # 检查是否进入响应数据部分
        if line == '### 响应数据':
            is_response_section = True
            continue

        # 解析响应数据表格
        if is_response_section and line.startswith('|'):
            parts = [part.strip() for part in line.split('|') if part.strip()]
            if len(parts) >= 3:
                field, type_, description = parts[0], parts[1], parts[2]
                field = field.strip('`')
                if 'number' in type_ or 'int' in type_:
                    type_ = 'number'
                elif 'message' in type_:
                    type_ = 'string | Lagrange.Send.Default[]'
                elif 'object' in type_:
                    if field == 'sender':
                        type_ = 'Sender'
                    else:
                        type_ = 'any'
                response_interface.append(f'    // {description.strip()}\n    {field}: {type_};\n')

    response_interface = ''.join(response_interface[2:])
    # 生成 TypeScript 接口
    interface_name = f'{snake_to_pascal(api_name)}Response'
    interface_content = f'export interface {interface_name} {{\n{response_interface}}}\n'

    return api_name, interface_content

def snake_to_pascal(snake_case_str):
    # 使用正则表达式将下划线分隔的单词拆分，并将每个单词的首字母大写
    pascal_case_str = ''.join(word.capitalize() for word in snake_case_str.split('_'))
    return pascal_case_str

# 将生成的 TypeScript 接口写入文件
def write_typescript_file(file_path, content):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

In [33]:
api_name, response_interface = parse_markdown(api_blocks[4])

print(f'API Name: {api_name}')
print(f'TypeScript Interface:\n{response_interface}')

API Name: get_msg
TypeScript Interface:
export interface GetMsgResponse {
    // 发送时间
    time: number;
    // 消息类型，同 [消息事件](../event/message.md)
    message_type: string;
    // 消息 ID
    message_id: number;
    // 消息真实 ID
    real_id: number;
    // 发送人信息，同 [消息事件](../event/message.md)
    sender: Sender;
    // 消息内容
    message: string | Lagrange.Send.Default[];
}



In [35]:
type_string = ''

for api_string in api_blocks:
    api_name, response_interface = parse_markdown(api_string)
    comment = '// response type for ' + api_name + '\n'
    type_string += comment
    type_string += response_interface + '\n'

open('type.ts', 'w', encoding='utf-8').write(type_string)

6300